In [63]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

In [4]:
db_conn = pg.connect(
    dbname="wheretolive",
    user="wheretolive",
    password="wheretolive",
    host="127.0.0.1")

In [79]:
sql_query = """
select bfs_nr_state.state, s.bfs_nr, s.min_income, s.rate as s_rate, m0c.rate as m0c_rate, m2c.rate as m2c_rate, m2c2s.rate as m2c2s_rate from
(select * from tax_rate
where profile = 'single') as s
join
(select * from tax_rate
where profile = 'married_no_children') as m0c
on m0c.bfs_nr = s.bfs_nr and m0c.min_income = s.min_income and m0c.max_income = s.max_income
join
(select * from tax_rate
where profile = 'married_2_children') as m2c
on m2c.bfs_nr = s.bfs_nr and m2c.min_income = s.min_income and m2c.max_income = s.max_income
join
(select * from tax_rate
where profile = 'married_2_children_2_salaries') as m2c2s
on m2c2s.bfs_nr = s.bfs_nr and m2c2s.min_income = s.min_income and m2c2s.max_income = s.max_income
join 
(select distinct bfs_nr, state from town) as bfs_nr_state
on bfs_nr_state.bfs_nr = s.bfs_nr
order by s.bfs_nr, s.min_income
"""

In [80]:
df = pd.read_sql(sql_query, db_conn)
df.head()

,bfs_nr,min_income,s_rate,m0c_rate,m2c_rate,m2c2s_rate
0,1,12500,0.593280,0.384000,0.384000,0.384000
1,1,15000,1.107467,0.320000,0.320000,0.320000
2,1,17500,1.618057,0.274286,0.274286,0.274286
3,1,20000,2.105500,0.407200,0.240000,0.240000
4,1,25000,2.829720,1.061440,0.192000,0.192000


In [46]:
df['2c_effect'] = df['m2c_rate'] - df['m0c_rate']
df['m_effect'] = df['m0c_rate'] - df['s_rate']
df['2s_effect'] = df['m2c2s_rate'] - df['m2c_rate']
df.head(n=24)

,state,bfs_nr,min_income,s_rate,m0c_rate,m2c_rate,m2c2s_rate,2c_effect,m_effect,2s_effect
0,ZH,1,12500,0.593280,0.384000,0.384000,0.384000,0.000000,-0.209280,0.000000
1,ZH,1,12500,0.593280,0.384000,0.384000,0.384000,0.000000,-0.209280,0.000000
2,ZH,1,15000,1.107467,0.320000,0.320000,0.320000,0.000000,-0.787467,0.000000
3,ZH,1,15000,1.107467,0.320000,0.320000,0.320000,0.000000,-0.787467,0.000000
4,ZH,1,17500,1.618057,0.274286,0.274286,0.274286,0.000000,-1.343771,0.000000
5,ZH,1,17500,1.618057,0.274286,0.274286,0.274286,0.000000,-1.343771,0.000000
6,ZH,1,20000,2.105500,0.407200,0.240000,0.240000,-0.167200,-1.698300,0.000000
7,ZH,1,20000,2.105500,0.407200,0.240000,0.240000,-0.167200,-1.698300,0.000000
8,ZH,1,25000,2.829720,1.061440,0.192000,0.192000,-0.869440,-1.768280,0.000000
9,ZH,1,25000,2.829720,1.061440,0.192000,0.192000,-0.869440,-1.768280,0.000000


In [47]:
df_effects = df.groupby(by=['min_income', 'state']).agg(
    {
        "2c_effect": [np.mean, np.std, np.var],
        "m_effect": [np.mean, np.std, np.var],
        "2s_effect": [np.mean, np.std, np.var]
    })
df_effects.head(n=24)

2c_effect                      m_effect                \
                      mean       std       var      mean           std   
min_income state                                                         
12500      AG     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           AI    -0.538759  0.032527  0.001058 -1.100291  6.642983e-02   
           AR     0.000000  0.000000  0.000000 -0.450033  3.005452e-02   
           BE     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           BL     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           BS     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           FR     0.000000  0.000000  0.000000 -0.376430  4.484609e-02   
           GE     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           GL     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           GR     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           JU     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           LU     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           NE     0.000000  0.000000  0.000000 -0.907066  2.165554e-02   
           NW     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           OW     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           SG     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           SH     0.000000  0.000000  0.000000 -0.087795  5.399894e-03   
           SO     0.000000  0.000000  0.000000  0.160000  5.367049e-17   
           SZ    -0.183856  0.028052  0.000787 -1.118459  1.706484e-01   
           TG     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           TI     0.000000  0.000000  0.000000  0.160000  0.000000e+00   
           UR     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           VD     0.000000  0.000000  0.000000  0.000000  0.000000e+00   
           VS     0.000000  0.000000  0.000000  0.000000  0.000000e+00   

                               2s_effect            
                           var      mean  std  var  
min_income state                                    
12500      AG     0.000000e+00       0.0  0.0  0.0  
           AI     4.412923e-03       0.0  0.0  0.0  
           AR     9.032739e-04       0.0  0.0  0.0  
           BE     0.000000e+00       0.0  0.0  0.0  
           BL     0.000000e+00       0.0  0.0  0.0  
           BS     0.000000e+00       0.0  0.0  0.0  
           FR     2.011172e-03       0.0  0.0  0.0  
           GE     0.000000e+00       0.0  0.0  0.0  
           GL     0.000000e+00       0.0  0.0  0.0  
           GR     0.000000e+00       0.0  0.0  0.0  
           JU     0.000000e+00       0.0  0.0  0.0  
           LU     0.000000e+00       0.0  0.0  0.0  
           NE     4.689624e-04       0.0  0.0  0.0  
           NW     0.000000e+00       0.0  0.0  0.0  
           OW     0.000000e+00       0.0  0.0  0.0  
           SG     0.000000e+00       0.0  0.0  0.0  
           SH     2.915886e-05       0.0  0.0  0.0  
           SO     2.880521e-33       0.0  0.0  0.0  
           SZ     2.912088e-02       0.0  0.0  0.0  
           TG     0.000000e+00       0.0  0.0  0.0  
           TI     0.000000e+00       0.0  0.0  0.0  
           UR     0.000000e+00       0.0  0.0  0.0  
           VD     0.000000e+00       0.0  0.0  0.0  
           VS     0.000000e+00       0.0  0.0  0.0

In [60]:
effects = set(df_effects.columns.get_level_values(0))
states = set(df_effects.index.get_level_values(1))
incomes = set(df_effects.index.get_level_values(0))
for effect in effects:
    for state in states:
        for min_income in incomes:
            df.loc[(df.min_income == min_income) & (df.state == state) , effect + '_est'] = df_effects[effect]['mean'][min_income][state]
df.head(n=24)

,state,bfs_nr,min_income,s_rate,m0c_rate,m2c_rate,m2c2s_rate,2c_effect,m_effect,2s_effect,m_effect_est,2c_effect_est,2s_effect_est
0,ZH,1,12500,0.593280,0.384000,0.384000,0.384000,0.000000,-0.209280,0.000000,-0.241382,0.000000,0.000000
1,ZH,1,12500,0.593280,0.384000,0.384000,0.384000,0.000000,-0.209280,0.000000,-0.241382,0.000000,0.000000
2,ZH,1,15000,1.107467,0.320000,0.320000,0.320000,0.000000,-0.787467,0.000000,-0.849317,0.000000,0.000000
3,ZH,1,15000,1.107467,0.320000,0.320000,0.320000,0.000000,-0.787467,0.000000,-0.849317,0.000000,0.000000
4,ZH,1,17500,1.618057,0.274286,0.274286,0.274286,0.000000,-1.343771,0.000000,-1.435075,0.000000,0.000000
5,ZH,1,17500,1.618057,0.274286,0.274286,0.274286,0.000000,-1.343771,0.000000,-1.435075,0.000000,0.000000
6,ZH,1,20000,2.105500,0.407200,0.240000,0.240000,-0.167200,-1.698300,0.000000,-1.796694,-0.188183,0.000000
7,ZH,1,20000,2.105500,0.407200,0.240000,0.240000,-0.167200,-1.698300,0.000000,-1.796694,-0.188183,0.000000
8,ZH,1,25000,2.829720,1.061440,0.192000,0.192000,-0.869440,-1.768280,0.000000,-1.870447,-0.929866,0.000000
9,ZH,1,25000,2.829720,1.061440,0.192000,0.192000,-0.869440,-1.768280,0.000000,-1.870447,-0.929866,0.000000


In [ ]:
df[()]

In [71]:
effects = set(df_effects.columns.get_level_values(0))
for effect in effects:
    mae = mean_absolute_error(df[effect], df[effect + '_est'])
    mse = mean_squared_error(df[effect], df[effect + '_est'])
    r2 = r2_score(df[effect], df[effect + '_est'])
    print('{}:\tMAE: {:.2f}\tMSE: {:.2f}\tR2: {:.2f}'.format(effect, mae, mse, r2))

m_effect:	MAE: 0.13	MSE: 0.04	R2: 0.99
2c_effect:	MAE: 0.09	MSE: 0.03	R2: 0.99
2s_effect:	MAE: 0.02	MSE: 0.00	R2: 0.97


In [75]:
#Sanity Check m2c_effect
df['m2c_effect'] = df['m2c_rate'] - df['s_rate']
df['m2c_effect_est'] = df['2c_effect_est'] + df['m_effect_est']
mae = mean_absolute_error(df['m2c_effect'], df['m2c_effect_est'])
mse = mean_squared_error(df['m2c_effect'], df['m2c_effect_est'])
r2 = r2_score(df['m2c_effect'], df['m2c_effect_est'])
print('m2c_effect:\tMAE: {:.2f}\tMSE: {:.2f}\tR2: {:.2f}'.format(mae, mse, r2))

m2c_effect:	MAE: 0.22	MSE: 0.11	R2: 0.99


In [76]:
#Sanity Check m2c2s_effect
df['m2c2s_effect'] = df['m2c2s_rate'] - df['s_rate']
df['m2c2s_effect_est'] = df['2c_effect_est'] + df['m_effect_est'] + df['2s_effect_est']
mae = mean_absolute_error(df['m2c2s_effect'], df['m2c2s_effect_est'])
mse = mean_squared_error(df['m2c2s_effect'], df['m2c2s_effect_est'])
r2 = r2_score(df['m2c2s_effect'], df['m2c2s_effect_est'])
print('m2c2s_effect:\tMAE: {:.2f}\tMSE: {:.2f}\tR2: {:.2f}'.format(mae, mse, r2))

m2c2s_effect:	MAE: 0.23	MSE: 0.12	R2: 0.99
